In [ ]:
# Sumani
# https://www.linkedin.com/in/sumanaruban/
# https://github.com/Sumanaruban
# 28-7-2024

# Simple Neural Network on Iris Dataset 🌸
This notebook demonstrates how to build a basic neural network using PyTorch to classify the Iris dataset.

### What you’ll learn:
- Basic neural network architecture
- How data is loaded and prepared
- How model training works
- How predictions are made and evaluated

In [ ]:
import torch
import torch.nn as nn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd

## 🌼 Iris Dataset Description
The Iris dataset contains 150 samples from 3 flower species:
- Setosa
- Versicolor
- Virginica

Each sample has 4 measurements:
- Sepal Length
- Sepal Width
- Petal Length
- Petal Width

Our goal is to classify the species based on these 4 features.

In [ ]:
# Load the Dataset
iris = load_iris()

In [ ]:
# Examine the dataset
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['target'] = iris.target
df

In [ ]:
# Examine the dataset
print (df['target'].value_counts())
print (iris.target_names)

In [ ]:
# Split the dataset for training and testing
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

In [ ]:
# Convert the dataset to tensor format
X_tr_tensor = torch.tensor(X_train, dtype=torch.float32)
y_tr_tensor = torch.tensor(y_train, dtype=torch.long)

## 🧠 What is a Neural Network?
A neural network is a function approximator made of layers of neurons:
- **Input Layer**: Takes input features (4 in this case)
- **Hidden Layer(s)**: Learns internal representations
- **Output Layer**: Produces class probabilities (3 classes here)

We’ll use PyTorch's `nn.Sequential` to define the model.

In [ ]:
class FullyConnectedNN(nn.Module):
    def __init__(self):
        super().__init__()

        # First Layer
        self.layer_1 = nn.Linear(4, 64)     # Input layer to 64 neurons
        self.activation_1 = nn.ReLU()       # ReLU activation function

        # Second Layer
        self.layer_2 = nn.Linear(64, 16)    # 64 neurons to 16 neurons
        self.activation_2 = nn.ReLU()       # Another ReLU

        # Third and the final Layer
        self.layer_3 = nn.Linear(16, 3)     # Output layer to 3 neurons (classes)

    def forward(self, x):
        # Forward pass
        x = self.layer_1(x)
        x = self.activation_1(x)
        x = self.layer_2(x)
        x = self.activation_2(x)
        x = self.layer_3(x)
        return x

In [ ]:
from torchsummary import summary

model = FullyConnectedNN()
summary(model, input_size=(1, 4)) 

In [ ]:
from torchviz import make_dot
import torch

model = FullyConnectedNN()
x = torch.randn(1, 4)
y = model(x)

make_dot(y, params=dict(model.named_parameters())).render("nn_graph", format="png")


## ⚙️ Training the Network
Each epoch involves:
1. **Forward pass**: Model makes predictions
2. **Loss computation**: Compare predictions with true labels
3. **Backward pass**: Compute gradients using backpropagation
4. **Optimizer step**: Adjust model weights

We repeat this for several epochs to minimize the loss.

In [ ]:
def train_model(model):    
    epochs = 400
    loss_arr = []

    # Use CrossEntropyLoss as our loss function.
    loss_fn = nn.CrossEntropyLoss()

    # Use Adam optimizer for adjusting the weights.
    optim = torch.optim.Adam(model.parameters(), lr=0.002)

    for epoch in range(epochs):
        # forward pass
        ypred = model(X_tr_tensor)

        # calculate loss
        loss = loss_fn(ypred, y_tr_tensor)
        loss_arr.append(loss.item())

        # backward pass
        loss.backward()
        optim.step()
        optim.zero_grad()
    
    # Plot the loss over epochs to visualize the learning process.
    plt.plot(loss_arr)
    plt.show()

In [ ]:
model = FullyConnectedNN()
train_model(model)

## ✅ Final Evaluation
After training, we use the test set to evaluate model performance.
We’ll check:
- Predictions vs actual values
- Accuracy of the classifier

This helps us assess how well the model has learned.

In [ ]:
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_pred = model(X_test_tensor)
y_test_pred = torch.argmax(y_test_pred, dim=1)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns

print("Accuracy:", accuracy_score(y_test_pred, y_test))

In [ ]:
confused_matrix = confusion_matrix(y_test_pred, y_test)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(confused_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=iris.target_names, yticklabels=iris.target_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()